# Cetacean Data set exploration

In [12]:
import duckdb

In [5]:
query = """
    SELECT
        interpreted."order",
        COUNT(*) as cnt
    FROM read_parquet('../../data/raw/occurrence/*.parquet')
    WHERE interpreted."order" IS NOT NULL
    GROUP BY interpreted."order"
    ORDER BY cnt DESC
    LIMIT 20;
"""

df = duckdb.query(query).to_df()
df

,order,cnt
0,Charadriiformes,12227884
1,Perciformes,12142554
2,Decapoda,10126273
3,Eupercaria incertae sedis,8971648
4,Gadiformes,7995036
5,Calanoida,6374922
6,Pleuronectiformes,6103701
7,Clupeiformes,5003369
8,Laminariales,4868612
9,Procellariiformes,2799529


In [7]:
query = """
    SELECT
        interpreted.family,
        COUNT(*) as cnt
    FROM read_parquet('../../data/raw/occurrence/*.parquet')
    WHERE interpreted."order" = 'Cetartiodactyla'
    GROUP BY interpreted.family
    ORDER BY cnt DESC;
"""

df = duckdb.query(query).to_df()
df

,family,cnt
0,Balaenopteridae,1089379
1,Phocoenidae,668114
2,Delphinidae,558034
3,Physeteridae,114233
4,Ziphiidae,70128
5,NaN,62733
6,Balaenidae,45681
7,Eschrichtiidae,26896
8,Monodontidae,7015
9,Kogiidae,2347


In [10]:
# Cetaceans are order "Cetacea" in the taxonomy
# This queries S3 directly — only pulls matching rows
query = """
    SELECT
        interpreted.scientificName,
        interpreted.decimalLatitude,
        interpreted.decimalLongitude,
        interpreted.eventDate,
        interpreted.date_year,
        interpreted.order,
        interpreted.family,
        interpreted.species,
        dropped,
        absence
    FROM read_parquet('../../data/raw/occurrence/*.parquet')
    WHERE interpreted.order = 'Cetartiodactyla'
    AND dropped IS NOT TRUE
    AND absence IS NOT TRUE
    AND interpreted.decimalLatitude BETWEEN 24 AND 49
    AND interpreted.decimalLongitude BETWEEN -130 AND -65;
"""

df = duckdb.query(query).to_df()

In [11]:
print(f"Shape: {df.shape}")
print(f"\nSpecies:\n{df['species'].value_counts().head(15)}")
print(f"\nYear range: {df['date_year'].min()} - {df['date_year'].max()}")

Shape: (460212, 10)

Species:
species
Megaptera novaeangliae        126765
Mesoplodon densirostris        51646
Tursiops truncatus             51222
Phocoena phocoena              27938
Balaenoptera physalus          24809
Eschrichtius robustus          23770
Delphinus delphis              17299
Balaenoptera musculus          13512
Balaenoptera acutorostrata     13038
Physeter macrocephalus         12755
Grampus griseus                 9756
Eubalaena glacialis             6529
Leucopleurus acutus             5860
Balaenoptera borealis           4010
Orcinus orca                    3770
Name: count, dtype: int64

Year range: 1849 - 2026


In [13]:
# Make sure directory exists
from pathlib import Path

# Save the filtered whale data

output_path = "../../data/raw/cetacean/us_cetacean_sightings.parquet"

Path(output_path).parent.mkdir(parents=True, exist_ok=True)

# Save
df.to_parquet(output_path, index=False)
print(f"Saved {len(df)} records to {output_path}")
print(f"File size: {Path(output_path).stat().st_size / 1e6:.1f} MB")

Saved 460212 records to ../../data/raw/cetacean/us_cetacean_sightings.parquet
File size: 8.7 MB
